In [22]:
import numpy as np
import math

###################### Calculate PSNR ######################
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [23]:
import math
import numpy as np
import cv2

###################### Calculate SSIM ######################
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [45]:
##################   Full-reference 평가   ##################

import torch
from PIL import Image
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import numpy as np
from torchmetrics import MultiScaleStructuralSimilarityIndexMeasure

# WSSDCNN은 이미지가 너무 작아서 따로 코드 삽입 
img1_path = Image.open('C://Users//knuvi//Desktop//AVSS//experiments//Fig//GT(16_9)//1.jpg')
#img1_path = Image.open('C://Users//knuvi//Desktop//AVSS//experiments//Fig//WSSDCNN//GT_LR//4.jpg')
img2_path = Image.open('C://Users//knuvi//Desktop//AVSS//experiments//Fig//HOP//1.jpg')


img1 = np.array(img1_path)
img2 = np.array(img2_path)

img1_reshape = np.reshape(img1,(4,1))
img2_reshape = np.reshape(img2,(4,1))

print(img1.shape)
print(img2.shape)
print(img2_reshape.shape)

#ms = ms_ssim(img1, img2, data_range=255, size_average=False)
ms_ssim = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0)
ms_ssim(img1_reshape,img2_reshape)

print(f'PSNR: {calculate_psnr(img1, img2)}')
print(f'SSIM: {calculate_ssim(img1, img2)}')
print(f'MS-SSIM: {ms_ssim}')

ValueError: cannot reshape array of size 6220800 into shape (4,1)

In [56]:
import matplotlib.pyplot as plt

from skimage import data
from skimage.color import rgb2gray
from skimage.morphology import disk
from skimage.filters.rank import gradient

from skimage import io

img = io.imread('C:/Users/knuvi/Desktop/AVSS/experiments/Fig/HOP/1.jpg')
img_gray = rgb2gray(img)

selection_element = skimage.morphology.disk(5) # matrix of n pixels with a disk shape

img_sharpness = gradient(img_gray, selection_element)

plt.imshow(img_sharpness, cmap="viridis")
plt.axis('off')
plt.colorbar()

plt.show()

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'